<a href="https://colab.research.google.com/github/1190303311/AI/blob/main/n_gram_maxent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import MaxentClassifier
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
nltk.download('movie_reviews')

from nltk.classify import MaxentClassifier
from nltk.corpus import movie_reviews

def word_feats(words):
 return dict([(word, True) for word in words])

negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)

trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
classifier = nltk.MaxentClassifier.train(trainfeats)

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(train_features, algorithm,max_iter=10)

classifier.show_most_informative_features(10)

#all_words = nltk.FreqDist(word for word in movie_reviews.words())
#top_words = set(all_words.keys()[:300])

def word_feats(words):
    return {word:True for word in words if word in top_words}

In [ ]:
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import MaxentClassifier
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.classify import MaxentClassifier
dataset = load_dataset("sst", "default")
train_features = dataset['train']
test_features = dataset['test']
feat_nltk = []
for k,y in zip(train_features['tokens'],train_features['label']):
  feat = {}
  if y>0.5: target=1 
  else: target=0
  for i in k.split('|'):
    feat[i] = True
  feat_nltk.append((feat,target))
algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(trainfeats, algorithm,max_iter=10)

In [ ]:
trainfeats[0]


In [ ]:
feats = []
for i in trainfeats:
  feat=[]
  feat.append(i[1])
  feat.extend(i[0].keys())
  feats.append(feat)
feats[0]

In [ ]:
import sys
import math
from collections import defaultdict
from datasets import load_dataset
import operator

class MaxEnt(object):
  def __init__(self):
    self.samples = []   #样本集合，（x,y)x是完整的一个样本
    self.Y = set([])   #标签
    self.numXY = defaultdict(int) #特征（x,y),count
    self.N = 0     #样本数量
    self.n = 0     #特征数量
    self.xyID = {}    #特征索引
    self.c = 0
    self.ep = []     #样本分布特征期望
    self.EP = []    #模型分布特征期望
    self.w = []    #n个特征的权值
    self.prew = []  #上一轮权值
    self.eps = 0.001

  def load_data_gram(self, features):
    for i,feat in enumerate(features):
      y = feat[0]
      X = feat[1:]
      self.samples.append(feat)
      self.Y.add(y)
      for x in X:
        self.numXY[(x,y)] += 1
    print(len(self.numXY))
    self.numXY = {(a,b):self.numXY[(a,b)] for a,b in self.numXY if self.numXY[(a,b)]>10 and self.numXY[(a,b)]<300}
    print(len(self.numXY))

  def initparams(self):
    self.N = len(self.samples)
    self.n = len(self.numXY)
    self.c = max([len(k)-1 for k in self.samples])
    self.w = [0.0]*self.n
    self.prew = self.w[:]
    self.sample_ep();

  def convergence(self):
    for w, lw in zip(self.w, self.prew):
      #print(math.fabs(w-lw))
      if math.fabs(w-lw) >=self.eps:
        return False
    return True;

  def sample_ep(self):
    self.ep = [0.0]*self.n
    for i, xy in enumerate(self.numXY):
      self.ep[i] = self.numXY[xy]*1.0/self.N
      self.xyID[xy] = i

  def zx(self, X):
    zx = 0.0
    for y in self.Y:
      sum = 0.0
      for x in X:
        if (x,y) in self.numXY:
          sum+=self.w[self.xyID[(x,y)]]
      zx += math.exp(sum)
    return zx

  def pyx(self, X):
    zx = self.zx(X)
    res = []
    for y in self.Y:
      sum = 0.0
      for x in X:
        if (x,y) in self.numXY:
          sum+=self.w[self.xyID[(x,y)]]
      yp = 1.0/ zx * math.exp(sum)
      res.append((y, yp))
    return res

  def model_ep(self):
    self.EP = [0.0]*self.n
    for sample in self.samples:
      X = sample[1:]
      pyx = self.pyx(X)
      for y, p in pyx:
        for x in X:
          if (x,y) in self.numXY:
            self.EP[self.xyID[(x,y)]]+=p*1.0/self.N
  
  def train(self, maxiter=10000):
    self.initparams()
    for i in range(maxiter):
      if i%50==0:
        print(self.w)
        print('iter: ',i,'...')
      self.prew = self.w[:]
      self.model_ep()
      for i, w in enumerate(self.w):
        self.w[i] += 1.0/self.c*math.log(self.ep[i]/self.EP[i])
      if self.convergence():
        break

  def predict(self, input):
    X = input.strip().split('\t')
    prob = self.pyx(X)
    return prob

maxent = MaxEnt()
maxent.load_data_gram(feats)
maxent.train()
test_features = negfeats[negcutoff:] + posfeats[poscutoff:]


In [ ]:
res = []
for feat in feats:
  res.append(maxent.pyx(feat[1:]))
predict=[]
for k in res:
  k = sorted(k, key=lambda x:x[1])
  target = k[-1][0]
  predict.append(target)
label=[]
for k in feats:
  label.append(k[0])
count=0
n = len(label)
for i in range(n):
  if label[i]==predict[i]:
    count+=1
print(count, n, count/n)

In [ ]:
feats = []
for i in test_features:
  feat=[]
  feat.append(i[1])
  feat.extend(i[0].keys())
  feats.append(feat)
res = []
for feat in feats:
  res.append(maxent.pyx(feat[1:]))
predict=[]
for k in res:
  k = sorted(k, key=lambda x:x[1])
  target = k[-1][0]
  predict.append(target)
label=[]
for k in feats:
  label.append(k[0])
count=0
n = len(label)
for i in range(n):
  if label[i]==predict[i]:
    count+=1
print(count, n, count/n)